## Price Data Scheduler

- Function Goal:
    * pull 48 hours of hourly price data once every hour 
- Tech:
    * python schedule library

In [3]:
# import dependencies
import pandas as pd
import numpy as np
import os
import requests
import json
import schedule
import time
import math
import datetime
from dotenv import load_dotenv


# import functions from other notebooks
from utils.utils import *
from utils.eval import *


Bad key text.latex.preview in file C:\Users\quint\anaconda3\envs\dev\envs\dev\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 123 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file C:\Users\quint\anaconda3\envs\dev\envs\dev\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 155 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.jpeg_quality in file C:\Users\quint\anaconda3\envs\dev\envs\dev\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 418 ('savefig.jpeg_quality: 95       # when a jpe

ModuleNotFoundError: No module named 'utils.trade'

In [6]:
load_dotenv()
kucoin_apikey=os.getenv('KUCOIN_API_KEY')

In [1]:
url = "https://api.kucoin.com/api/v1/market/orderbook/level1?symbol=SUSD-USDT"
response = requests.get(url).json()
response["data"]["price"]

NameError: name 'requests' is not defined

In [8]:
def initialize_kucoin_prices(sma_window, coin_list, kucoin_apikey, base_currency='USD')
    '''
    Uses Kucoin's API to create a large dictionary of ohlcv recent prices of a list of crypto coins
    
     Parameters:
    
    sma_window (int) : window of time in hours for pricing to be found
    coin_list (list) : a list containing strings of coins to find pricing data (ex: ['BTC','ETH','USDT'])
    api_key (string) : API key from Kucoin.com
    base_currency (string) : (='USD' default) Base currency for pricing data.    
   
    Returns:
    
    ohlcv_prices (dict) : A dictionary containing all ohlcv pricing data for all currencies.
    '''
    # Initialize date request
    current_date = datetime.now()
    end_date = current_date - timedelta(hours=sma_window)
    end_date = end_date.strftime('%d/%m/%Y %H:%M:%S')
    
    # Initialize Dictionary
    ohlcv_prices = {}
    
    for coin in coin_list:
        historical_prices = get_recent_prices(coin,
                                              base_currency,
                                              end_date,
                                              kucoin_apikey)
        ohlcv_prices.update(historical_prices)
    return ohlcv_prices


def get_kucoin_ohlcv(currency, sma_window, df=None):          
    global kucoin_apikey
    
    #Initialize timestamps
    step_size = 500*3600
    end_ts = int(datetime.now().timestamp())
    start_ts = int(datetime.strptime('2021-01-01 00:00:00', '%Y-%m-%d %H:%M:%S').timestamp())
    
    if df is None:
        df = pd.DataFrame()
    sma_window = sma_window-2
    
    rows = df.shape[0]
    if rows > 1:
        updated_entries = sma_window - rows +2
        df_append = initialize_prices(updated_entries,
                       [currency],
                       kucoin_apikey)[currency]
        if(df_append.iloc[-1,0] == df.iloc[-1,0]):
            print("dataframe is up-to-date")
        else:
            df = df.iloc[1: , :]
            print("popping oldest record")


            df = df.append(df_append.iloc[-1,:],
                          ignore_index=True)
            print(f"getting {updated_entries+1} {currency} records")
    else:
        df = initialize_kucoin_prices(sma_window,
                           [currency],
                           kucoin_apikey)[currency]
        print(f"getting {sma_window+2} {currency} records")
    
    
    return df
    
    
    
"""
set job to run on the 1st second of the 1st minute of every hour
"""
schedule.every().hour.at('00:01').do(get_kucoin_ohlcv)


while True:
    schedule.run_pending()
    time.sleep(1)

SyntaxError: invalid character in identifier (3500904028.py, line 8)